In [12]:
%load_ext autoreload
%autoreload 2

import os

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
# print(physical_devices)
assert len(physical_devices) == 1, "Did not see the expected number of GPUs"
# to allow other tensorflow processes to use the gpu
# https://stackoverflow.com/a/60699372/7989988
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import numpy as np
from tensorflow import keras
from tensorflow.keras import Model, Input, layers
from IPython.display import display
import tensorflow_datasets as tfds
import time
import matplotlib.pyplot as plt
import enlighten
import tensorflow_probability as tfp
# from dotmap import DotMap
from box import Box as DotMap
from icecream import ic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AssertionError: Did not see the expected number of GPUs

In [2]:
tf.constant([1., 2, 3]) < 3

2022-03-10 19:18:16.941640: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-03-10 19:18:16.941666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: maxeonyx-pc-manjaro
2022-03-10 19:18:16.941670: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: maxeonyx-pc-manjaro
2022-03-10 19:18:16.941730: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.54.0
2022-03-10 19:18:16.941743: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.54.0
2022-03-10 19:18:16.941746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.54.0
2022-03-10 19:18:16.942681: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in p

<tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True, False])>

In [3]:
import models
import training
import datasets
import vizualization
import schedules
import create_dataset

# todo
# - combine shuffled and unshuffled datasets into same thing
# - add extra shuffled indices and noise
# - add "noise fraction" parameter which can be changed the same way as the "n" parameter.
# - add different amounts of noise to the data and see what happens

In [4]:
config = DotMap({
    'ds': 'hands',
    'distributed': False,
    'minibatch_size': 24,
    'n_steps': 5,
    'test_size': 300,
    'test_minibatch_size': 25,
    'test_interval': 500,
    'test_n_shuf': [392, 1, 64, 128, 256],
    'test_n_seq': [392, 1, 128, 256, 512],
    'test_autoregressive': False,
    'display_images': True,
    'display_image_interval': 500,
    'dont_display_until_loss': 0.48,
    'bg_color': [1.0, 0.4, 0.6],
    'lr_schedule': ['constant', 0.0004],
    'lr_warmup': 100,
    'grad_accum_steps': None, #['exponential', 1, 4],
    'max_accum_steps': 4,
    'use_wandb': True,
    'wandb_log_interval': 10,
    'loss_window_size': 80,
    'kmeans_batch_size': 1000,
    'mixed_float': False,
})

# need to change for multiworkerstrategy
if config.distributed:
    config.num_devices = len(physical_devices)
else:
    config.num_devices = 1
config.global_batch_size = config.minibatch_size * config.num_devices

In [5]:
# idea is to not have to re-run this cell for subsequent training runs unless the centroids change

ds_configs = DotMap({
    'mnist': {
        'type': 'image',
        'tfds_name': 'mnist',
        'continuous': False,
        'buffer_size': 60000,
        'noise_fraction': None,
        'shuffle': True,
        'n_split_distribution': 'mnist_gamma',
        'n_colors': 4,
        'n_color_dims': 1,
        'image_size': (28, 28),
    },
    'mnist_7x7': {
        'type': 'image',
        'tfds_name': 'mnist',
        'continuous': False,
        'buffer_size': 60000,
        'noise_fraction': None,
        'shuffle': True,
        'n_split_distribution': 'mnist_7x7_gamma',
        'n_colors': 4,
        'n_color_dims': 1,
        'rescale': (7, 7),
    },
    'mnist_7x7_contin': {
        'type': 'image',
        'tfds_name': 'mnist',
        'continuous': True,
        'loss': 'gaussian',
        'n_dist_params': 2, # gaussian dist has 2 params
        'buffer_size': 60000,
        'noise_fraction': None,
        'shuffle': True,
        'n_split_distribution': None,
        'n_colors': 4,
        'n_color_dims': 1,
        'rescale': (7, 7),
    },
    'mnist_binary_7x7': {
        'type': 'image',
        'tfds_name': 'mnist',
        'continuous': False,
        'buffer_size': 60000,
        'noise_fraction': None,
        'shuffle': True,
        'n_split_distribution': 'mnist_7x7_gamma',
        'n_colors': 2,
        'n_color_dims': 1,
        'rescale': (7, 7),
    },
    'celeb': {
        'type': 'image',
        'tfds_name': 'celeb_a',
        'continuous': False,
        'buffer_size': 10000,
        'noise_fraction': None,
        'shuffle': True,
        'n_colors': 16,
        'n_split_distribution': 'mnist_gamma',
        'n_color_dims': 3,
        'image_size': (218, 178),
        'rescale': (32, 39),
    },
    'hands': {
        'type': 'hands',
        'continuous': True,
        'loss': 'gaussian',
        'n_dist_params': 2, # gaussian dist has 2 params
        'shuffle': 'joints',
        'n_dof': 23,
        'n_frames': 60,
        'n_split_distribution': None,
        'buffer_size': 10000,
        'noise_fraction': None,
    },
})

config.dataset = ds_configs[config.ds]
config.dataset.discrete = not config.dataset.continuous

if config.dataset.type == 'image':
    dataset, metadata = tfds.load(config.dataset.tfds_name, with_info=True, as_supervised=True)
    ds_train_original = dataset['train'].map(datasets.ignore_label)
    ds_test_original = dataset['test'].map(datasets.ignore_label)
    centroids = datasets.find_centroids(config, ds_train_original)
    if config.dataset.rescale:
        config.dataset.image_size = config.dataset.rescale
    config.dataset.seq_length = config.dataset.image_size[0]*config.dataset.image_size[1]*config.dataset.n_color_dims
elif config.dataset.type == 'hands':
    dataset = create_dataset.tf_dataset()
    centroids = None
    ds_train_original = dataset.map(datasets.ignore_metadata)
    ds_test_original = dataset.map(datasets.ignore_metadata)
    config.dataset.seq_length = config.dataset.n_dof * config.dataset.n_frames
    
if config.dataset.n_split_distribution == 'mnist_gamma':
    gamma_dist, gamma_name = datasets.mnist_gamma_distribution()
    datasets.plot_distribution(config, gamma_dist, gamma_name)
elif config.dataset.n_split_distribution == 'mnist_7x7_gamma':
    gamma_dist, gamma_name = datasets.gamma_distribution_7x7()
    datasets.plot_distribution(config, gamma_dist, gamma_name)
elif config.dataset.n_split_distribution == 'mnist_12x12_gamma':
    gamma_dist, gamma_name = datasets.gamma_distribution_12x12()
    datasets.plot_distribution(config, gamma_dist, gamma_name)
else:
    gamma_dist, gamma_name = None, None


ds = datasets.Datasets(config, ds_train_original, ds_test_original, centroids, gamma_dist)
viz = vizualization.Viz(config, ds, centroids)
ds_train, ds_test = ds.make_datasets()


Not using gradient accumulation


In [6]:
if config.dataset.type == 'image':
    display_colors, display_idxs, *_ = next(iter(ds_train))
    if config.grad_accum_steps:
        display_colors,display_idxs = display_colors[0],display_idxs[0]
    if config.dataset.continuous:
        display_colors = ds.reinvent_color_dim(display_colors)
    viz.showSeq(display_colors, display_idxs, config.dataset.image_size, do_unquantize=config.dataset.discrete, max_images=9)

In [7]:
if config.dataset.type == 'image':
    x_idx = tf.range(784)
    pos_enc = models.pos_enc(n_dims=16, scale=100)
    x = models.dual_positional_encoding((28,28), pos_enc)(x_idx)
    x = x / 2. + 0.5
    x = tf.expand_dims(tf.transpose(x), -1)
    x_idx = tf.tile(x_idx[None, :], [16, 1])
    viz.showSeq(x[:8], x_idx[:8], (28, 28), 8)
    viz.showSeq(x[8:16], x_idx[8:16], (28, 28), 8)
    viz.showSeq(x[16:24], x_idx[16:24], (28, 28), 8)
    viz.showSeq(x[24:], x_idx[24:], (28, 28), 8)

In [8]:


# model config
config.model = DotMap({
    'discrete': config.dataset.discrete,
    'continuous': config.dataset.continuous,
    'n_enc_a_layers': 1,
    'n_enc_b_layers': 1,
    'ffl_dim': 64,
    'embd_dim': 16,
    'n_dec_layers': 1,
    'dec_dim': 400,
    'n_heads': 8,
    'dropout_rate': 0.1,
    'use_idxs_input': True,
    'architecture': 'anp',
    'use_relative_positions': False,
    'activation': 'swish'
})

if config.dataset.type == 'image':
    config.model.n_colors = config.dataset.n_colors
    config.model.n_color_dims = config.dataset.n_color_dims
    config.model.image_size = config.dataset.image_size
    config.model.seq_len = config.dataset.image_size[0] * config.dataset.image_size[1]
    config.model.position_embedding = 'pos_enc'
else:
    config.model.seq_len = config.dataset.n_frames * config.dataset.n_dof
    config.model.n_frames = config.dataset.n_frames
    config.model.n_dof = config.dataset.n_dof
    config.model.position_embedding = 'pos_and_embd'
    

if config.dataset.continuous:
    config.model.n_dist_params = config.dataset.n_dist_params

if config.distributed == False:
    strategy = tf.distribute.get_strategy()
else:
    strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = models.transformer(config.model)
    # Adam params taken from the linked notebook
    optimizer = keras.optimizers.Adam(learning_rate=schedules.learning_rate_schedule(config))

if config.distributed:
    ds_train = strategy.experimental_distribute_dataset(ds_train)

config.training_mode = 'query_next'

import socket
model_name = models.model_name(config)
print(model_name)
# model.summary()

maxeonyx-pc-manjaro-hands-contin-bs1x1x24


In [9]:
evaler = training.Evaluator(config, model, optimizer, viz, ds, ds_train, ds_test)

training_loop = training.TrainingLoop(config, evaler, model_name)

In [10]:


with strategy.scope():
    training_loop.train()

KeyboardInterrupt: 

In [ ]:
model.save(f"models/{model_name}")

In [ ]:
evaler.process_batch(show_input=True)
evaler.new_test_batch()
evaler.process_batch(show_input=True)

In [ ]:
manager = enlighten.get_manager()

n_steps = 8000*23
counter = manager.counter(total=n_steps)


dof_idxs = tf.tile(tf.range(23)[None, :], [59, 1])
frame_idxs = tf.tile(tf.range(59)[:, None], [1, 23]) * config.model.n_dof
idxs = dof_idxs + frame_idxs
idxs = tf.reshape(idxs, [-1])

def iteration(i, data):
    tar_idxs = (59 * 23 + i % 23)[None]
    inp = data[-59*23:]
    result = model([inp[None, :], idxs[None, :], tar_idxs[None, :], models.MASK_NONE, models.MASK_NONE])
    result = tf.reshape(result, [1, 1, 2])
    dist = tfp.distributions.Normal(loc=result[:, 0, 0], scale=result[:, 0, 1])
    sample = dist.sample()
    data = tf.concat([data, sample], axis=0)
    return i+1, data
         


@tf.function(
    input_signature=[tf.TensorSpec([None])],
)
def do_230(data):
    
    return tf.while_loop(
        cond=lambda i, data: i < 23,
        body=iteration,
        loop_vars=[
            tf.constant(0),
            data
        ],
        shape_invariants=[
            tf.TensorShape([]),
            tf.TensorShape([None]),
        ],
    )


inp_data = tf.random.uniform([59*23], minval=-1, maxval=1, dtype=tf.float32)
out_data = inp_data
while i < n_steps:
    i, result = do_230(out_data)
    out_data = tf.concat([out_data, result], axis=0)
    counter.update(incr=100)

counter.close()

print(out_data)

In [ ]:
import create_dataset
create_data

#### 